In [1]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
from tqdm import tqdm
import emoji
from sklearn.metrics import *

In [32]:
def func_Eval(Y_Actual,Y_pred):

    TrueNegative=0
    TruePositive=0
    FalsePositive=0
    FalseNegative=0
    for i in range(len(Y_pred)):
        #if the actual class is negative
        if(Y_Actual[i]=="fake"):
            
            #if the predicited class is negative
            if(Y_pred[i]=="fake"):
                TrueNegative+=1
                
            #if the predicted class is positive
            else:
                FalsePositive+=1
                
        #if the actual class is positive
        else:
            #if the predicited class is positive
            if(Y_pred[i]=="real"):
                TruePositive+=1
                
            #if the predicited class is negative
            else:
                FalseNegative+=1

    Confusion_Matrix=[[TrueNegative,FalsePositive],[FalseNegative,TruePositive]]
    
    Confusion_Matrix=pd.DataFrame(Confusion_Matrix,columns=['Predicted Negative','Predicted Positive'])
    
    Confusion_Matrix.rename(index={0: "Actual Negative", 1: "Actual Positive"},inplace=True)
    
    MyPrecision=TruePositive/(TruePositive+FalsePositive)

    MyRecall=TruePositive/(TruePositive+FalseNegative)
    
    MyAccuracy=(TruePositive+TrueNegative)/(TruePositive+TrueNegative+FalseNegative+FalsePositive)
    
    MyF1score= 2*(MyPrecision*MyRecall)/(MyPrecision+MyRecall)
    
    MyPrecisionZero=TrueNegative/(TrueNegative+FalseNegative)

    MyRecallZero=TrueNegative/(TrueNegative+FalsePositive)
    
    
    MyF1scoreZero= 2*(MyPrecisionZero*MyRecallZero)/(MyPrecisionZero+MyRecallZero)

    
    print("-----------------")
    print("Accuracy:",MyAccuracy)
    print("-----------------")

    print()

    print("For the Positive CLass, Label:4")
    print("-----------------")
    print("Precision:",MyPrecision)

    print("-----------------")
    print("Recall:",MyRecall)

    print("-----------------")
    print("F1 Score:",MyF1score)

    print()

    print("For the Negative CLass, Label:0")

    print("-----------------")
    print("Precision:",MyPrecisionZero)

    print("-----------------")
    print("Recall:",MyRecallZero)

    print("-----------------")
    print("F1 Score:",MyF1scoreZero)

    print()
    print("--------")
    print("Macro Average F1",(MyF1score+MyF1scoreZero)/2)
    print("-----------------")
    print("Confusion Matrix:")
    print(Confusion_Matrix)

In [33]:
def preprocess_text_caller(Tweets_List):
    stop_words_list=stopwords.words('english')
    for i in (range(len(Tweets_List))):
        Tweets_List[i]=preprocess_text(Tweets_List[i],stop_words_list=stop_words_list)

In [34]:
def preprocess_text(tweet, flag_stemm=True, flag_lemm=False, stop_words_list=None):
#     print(tweet)
    tweet=emoji.demojize(tweet)
    tweet=tweet.replace(":"," ")
#     print(tweet)

    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
    ## Tokenize (convert from string to list) and remove the stop words
#     print(tweet)

    tokenize_tweet = tweet.split()

    if stop_words_list is not None:
        tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]


    ## Stemming (remove -ing, -ly, ...)
    if flag_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

    ## Lemmatisation (convert the word into root word)
    if flag_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
#     print(tokenize_tweet)
    ## back to string from list
    tweet = " ".join(tokenize_tweet)
    return tweet

In [35]:
df_training=pd.DataFrame(pd.read_csv("Constraint_English_Train - Sheet1.csv"))
df_training.drop(columns="id",inplace=True)
df_training
df_training['tweet_original']=df_training['tweet']
preprocess_text_caller(df_training['tweet'])



# df_testing=pd.DataFrame(pd.read_csv("testset-levela.tsv",sep="\t"))
# df_testing['tweet_original']=df_testing['tweet']
# preprocess_text_caller(df_testing['tweet'])


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range parameter (1,2) means that unigram and bigram will be taken
# Count Vecotrizer automatically preprocess the tweets
vectorizer = TfidfVectorizer(ngram_range=(1,4),min_df=5)

In [37]:
Train_ngrams=vectorizer.fit_transform(df_training.tweet)


In [38]:
df_validation=pd.DataFrame(pd.read_csv("Constraint_English_Val - Sheet1.csv"))
df_validation.drop(columns="id",inplace=True)
df_validation
df_validation['tweet_original']=df_validation['tweet']
preprocess_text_caller(df_validation['tweet'])


In [39]:
Validation_ngrams=vectorizer.transform(df_validation.tweet)


In [40]:
Validation_ngrams

<2140x5791 sparse matrix of type '<class 'numpy.float64'>'
	with 40205 stored elements in Compressed Sparse Row format>

In [41]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf.fit(Train_ngrams,df_training["label"])
y_pred=clf.predict(Validation_ngrams)


In [42]:
f1_score(y_pred,df_validation["label"],average="macro")

0.918082612536882

In [43]:
accuracy_score(y_pred,df_validation["label"])

0.9182242990654206

In [44]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
clf.fit(Train_ngrams,df_training["label"])
y_pred=clf.predict(Validation_ngrams)


In [45]:
accuracy_score(y_pred,df_validation["label"])

0.9182242990654206

In [46]:
func_Eval(y_pred,df_validation["label"])

-----------------
Accuracy: 0.9182242990654206
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.9223214285714286
-----------------
Recall: 0.9214986619090099
-----------------
F1 Score: 0.9219098616688978

For the Negative CLass, Label:0
-----------------
Precision: 0.9137254901960784
-----------------
Recall: 0.914622178606477
-----------------
F1 Score: 0.91417361451692

--------
Macro Average F1 0.9180417380929089
-----------------
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                 932                  87
Actual Positive                  88                1033
